In [45]:
import nltk
import time
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from transformers import pipeline
import torch
from nltk.tokenize import word_tokenize
from sklearn.metrics import mean_squared_error, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from sklearn.svm import SVR, SVC
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import OneHotEncoder

print('Job done-')

Job done-


#### 1. BERT 준비 ####

In [46]:
# 특정 단어에 대한 임베딩 얻기
def get_embbeding_of_a_word(target_word: str, tokenizer, outputs, tokens):
    try:
        word_index = tokens['input_ids'][0].tolist().index(tokenizer.convert_tokens_to_ids(target_word))
    
        return outputs.last_hidden_state[:, word_index, :]
    except Exception as exp:
        return None
    
# 완료 표시
print('Job done')

Job done


#### 2. 워드 임베딩 ####

In [47]:
total_file_path = "data/dataset.txt"
total_documents = []
total_document = []

vector_size = 768

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

with open(total_file_path, 'r', encoding='utf-8') as file:
    # Postmortum 읽기
    for line in file:
        # Document separation
        if line == '\n':
            total_documents.append('\n'.join(total_document))
            total_document = []

        # Add to document
        total_document.append(line)

    # For last game document
    total_documents.append('\n'.join(total_document))

##print(total_documents)

# 워드 임베딩
total_document_vectors = np.array([]).reshape(0, vector_size)
for total_document in total_documents:
    document_vectors = np.array([]).reshape(0, vector_size)
    
    # tokenizing
    tokens = tokenizer('\n'.join(total_document),
                       return_tensors='pt',
                       max_length=512)
    
    # Word embedding
    outputs = model(**tokens)
        
    # Feature vector of a document (game)
    document_vectors = outputs.last_hidden_state.mean(axis=1)
    
    # Append to total document vector
    total_document_vectors = np.vstack((total_document_vectors, document_vectors.detach().numpy()))
    
# Debug
print('Job done')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max 

Job done


#### 3. 목표값 붙이기

In [48]:
file_path = 'data/classification.txt'
class_list = []

class_size = 28   ### 29 -> 28
with open(file_path, 'r') as file:
    for line in file:
        class_type = str(line.strip())
        class_list.append(class_type)
        
print(class_list)

ohe = OneHotEncoder(sparse=False)
class_list = ohe.fit_transform(np.array(class_list).reshape(-1, 1))

label_size = class_list.shape[1]
data_index = np.arange(1, total_document_vectors.shape[0] + 1).reshape(-1, 1)
data = np.hstack((data_index, total_document_vectors, class_list))

#debug
print('jobs finished b')

['unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'design', 'tool', 'time', 'unrelated', 'planning', 'planning', 'unrelated', 'planning', 'planning', 'unrelated', 'communication', 'communication', 'communication', 'unrelated', 'unrelated', 'planning', 'planning', 'unrelated', 'optimization', 'optimization', 'unrelated', 'planning', 'planning', 'unrelated', 'unrelated', 'communication', 'communication', 'communication', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'testing', 'testing', 'unrelated', 'documentation', 'planning', 'documentation', 'documentation', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'team', 'team', 'time', 'planning', 'team', 'team', 'team', 'team', 'team', 'team', 'team', 'planning', 'planning', 'team', 'unrelated', 'team', 'team', 'team', 'team', 'team', 'team', 'team', 'unrelated', 'team', 't

/home/hooni/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [55]:
test_ratio = 0.2

# 데이터 준비
X_train, X_test, y_train, y_test = train_test_split(data[:, :vector_size + label_size],
                                                    data[:, vector_size + label_size],
                                                    test_size=test_ratio,
                                                    stratify=data[:, vector_size + label_size],
                                                    random_state=0)

# 레이블 분리
X_train_index, X_test_index = X_train[:, 0], X_test[:, 0]
X_train, X_test = X_train[:, 1:], X_test[:, 1:]

pca_var_ratio = 0.99
pca = PCA(n_components=pca_var_ratio)
original_dimension = X_train.shape[1]
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print(f'Reduced Dimension: {original_dimension} -> {X_train.shape[1]}')

# Result Preparation
result_table = pd.DataFrame(columns=['index', 'type', 'model', 'truth', 'prediction', 'f1', 'auc'])

Reduced Dimension: 781 -> 133


#### 4. 모델 학습

##### Experiment Design
1. Word Embedding
- TF-IDF
- Word2Vec
- Bert

2. Classfier
- Logistic Regression
- SVM (Support Vector Machine)
- RF (Random Forest)
- XGBoost (Gradient Boosting Machine)

4-1 LR

In [50]:
# LR 학습
timer = time.time()
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
print(f'Elapsed time for LR training: {(time.time() - timer)/60:0.2f} min')

# LR 학습데이터 추론
print('[LR 학습 데이터]')
train_prediction = lr_model.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')
f1 = f1_score(y_train, train_prediction, average='weighted')
auc = roc_auc_score(y_train, train_prediction)
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index], 'train', 'LR', y_train[index], train_prediction[index],
                                               f1_score(y_train, train_prediction, average='weighted'),
                                               roc_auc_score(y_train, train_prediction)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# LR 추론
print('[LR 검증 데이터]')
test_prediction = lr_model.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')
f1 = f1_score(y_test, test_prediction, average='weighted')
auc = roc_auc_score(y_test, test_prediction)
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index], 'test', 'LR', y_test[index], test_prediction[index],
                                               f1_score(y_test, test_prediction, average='weighted'),
                                               roc_auc_score(y_test, test_prediction)]
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

Elapsed time for LR training: 0.00 min
[LR 학습 데이터]
y_train=array([1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 1.

4-2 SVC

In [54]:
param_space = {
    'C': (1, 10000),
    'gamma': (0.01, 1.0)
}

# 하이퍼 파라미터 튜닝
svc_rbf = SVC(kernel='rbf')
opt = BayesSearchCV(svc_rbf, param_space, n_iter=50, cv=10, n_jobs=-1, n_points=8, verbose=0)

timer = time.time()
opt.fit(X_train, y_train)
print(f'Elapsed time for SVC optimization: {(time.time() - timer)/60:0.2f} min')
print("최적의 하이퍼파라미터:", opt.best_params_)

# SVC 학습
timer = time.time()
svc_rbf = SVC(kernel='rbf', **opt.best_params_)
svc_rbf.fit(X_train, y_train)
print(f'Elapsed time for SVC training: {(time.time() - timer)/60:0.2f} min')

# SVC 학습데이터 추론
print('[SVC 학습 데이터]')
train_prediction = svc_rbf.predict(X_train)
print(f'{y_train=}')
print(f'{train_prediction=}')
f1 = f1_score(y_train, train_prediction, average='weighted')
auc = roc_auc_score(y_train, train_prediction) ####################one_hot_encoding 제외
for index in range(train_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_train_index[index], 'train', 'SVC', y_train[index], train_prediction[index],
                                               f1_score(y_train, train_prediction, average='weighted'),
                                               roc_auc_score(y_train, train_prediction)]  ######one_hot_encoding 제외
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

# SVC 추론
print('[SVC 검증 데이터]')
test_prediction = svc_rbf.predict(X_test)
print(f'{y_test=}')
print(f'{test_prediction=}')
f1 = f1_score(y_test, test_prediction, average='weighted')
auc = roc_auc_score(y_test, test_prediction) ######one_hot_encoding 제외
for index in range(test_prediction.shape[0]):
    result_table.loc[result_table.shape[0]] = [X_test_index[index], 'test', 'SVC', y_test[index], test_prediction[index],
                                               f1_score(y_test, test_prediction, average='weighted'),
                                               roc_auc_score(y_test, test_prediction)] ######one_hot_encoding 제외
print(f'result: {f1=:0.3f}, {auc=:0.3f}')

/home/hooni/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/hooni/anaconda3/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Elapsed time for SVC optimization: 0.19 min
최적의 하이퍼파라미터: OrderedDict([('C', 230), ('gamma', 0.01)])
Elapsed time for SVC training: 0.00 min
[SVC 학습 데이터]
y_train=array([1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1.,

4-3 Random Forest

In [52]:
#X_train, X_test, y_train, y_test = train_test_split(data[:, :vector_size + label_size],
 #                                                   data[:, vector_size + label_size],
  #                                                  test_size=test_ratio,
   #                                                 stratify=data[:, vector_size + label_size],
    #                                                random_state=0)


# 필요한 라이브러리 임포트
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 예제 데이터 생성 (여러분은 실제 데이터를 사용해야 합니다.)
# X는 문단들의 리스트, y는 해당 문단들의 클래스 레이블
#X = ["문단 1", "문단 2", ..., "문단 N"]
#y = [1, 2, ..., 15]

# TF-IDF 벡터화
#vectorizer = TfidfVectorizer()
#X_tfidf = vectorizer.fit_transform(X)

# 학습 데이터와 테스트 데이터로 나누기
#X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Random Forest 모델 생성 및 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = rf_model.predict(X_test)

# 정확도 및 분류 보고서 출력
accuracy = accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred, multi_class='ovr')  # 'ovr'은 One-vs-Rest 방식을 사용함
print(f'AUC: {auc_score:0.3f}')
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

AUC: 0.856
Accuracy: 0.87

Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      0.93      0.89        75
         1.0       0.90      0.78      0.84        59

    accuracy                           0.87       134
   macro avg       0.87      0.86      0.86       134
weighted avg       0.87      0.87      0.86       134



####################

4-4 XGBoost

In [53]:
# 필요한 라이브러리 임포트
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# 예제 데이터 생성 (여러분은 실제 데이터를 사용해야 합니다.)
# X는 문단들의 리스트, y는 해당 문단들의 클래스 레이블
#X = ["문단 1", "문단 2", ..., "문단 N"]
#y = [1, 2, ..., 15]

# TF-IDF 벡터화
#vectorizer = TfidfVectorizer()
#X_tfidf = vectorizer.fit_transform(X)

# 학습 데이터와 테스트 데이터로 나누기
#X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# XGBoost 모델 생성 및 학습
xgb_model = XGBClassifier(objective="multi:softmax", num_class=15, random_state=42)
xgb_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = xgb_model.predict(X_test)

# 정확도 및 분류 보고서 출력
accuracy = accuracy_score(y_test, y_pred)
auc_score = roc_auc_score(y_test, y_pred, multi_class='ovr')  # 'ovr'은 One-vs-Rest 방식을 사용함
print(f'AUC: {auc_score:0.3f}')
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

AUC: 0.955
Accuracy: 0.96

Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.96      0.96        75
         1.0       0.95      0.95      0.95        59

    accuracy                           0.96       134
   macro avg       0.95      0.95      0.95       134
weighted avg       0.96      0.96      0.96       134



In [ ]:
result_table.to_excel('result/result_table_bert.xlsx', index=False)

pd.pivot_table(result_table,
               values=['f1', 'auc'],
               index=['type', 'model'],
               aggfunc=['mean', 'std'])

In [ ]:
print(y_train.shape)
print(train_prediction.shape)

In [ ]:
pip install torch
pip install transformers
pip install datasets
pip install scikit-learn

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# 데이터셋 로드
dataset = load_dataset("nsmc")

# 데이터셋 전처리
class NSMCDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 학습 및 테스트 데이터 분할
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset['train']['document'],
    dataset['train']['label'],
    test_size=0.2,
    random_state=42
)

# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 데이터셋 인코딩
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# PyTorch Dataset 및 DataLoader 생성
train_dataset = NSMCDataset(train_encodings, train_labels)
test_dataset = NSMCDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# 모델 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 테스트
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).tolist()
        all_preds.extend(preds)
        all_labels.extend(labels.tolist())

# 정확도 및 분류 보고서 출력
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)